# 03 Model Calibration
This notebook focuses on calibrating valuation and risk models using historical and market data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## Load Historical Data

In [ ]:
# Example: load equity returns and factor returns (CSV placeholders)
equities = pd.read_csv('./data/equity_returns.csv', parse_dates=['date'])
factors = pd.read_csv('./data/factor_returns.csv', parse_dates=['date'])

df = pd.merge(equities, factors, on='date')
df.head()

## Factor Model Calibration (Example: CAPM)

In [ ]:
y = df['equity_return'].values.reshape(-1,1) # type: ignore
X = df[['market_return']].values

model = LinearRegression().fit(X, y)
beta = model.coef_[0][0]
alpha = model.intercept_[0] # type: ignore

preds = model.predict(X)
r2 = r2_score(y, preds)
mse = mean_squared_error(y, preds)

print(f"Alpha: {alpha:.4f}, Beta: {beta:.4f}, R^2: {r2:.3f}, MSE: {mse:.5f}")

## Visualize Fit

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(X, y, alpha=0.5, label='Observed')
plt.plot(X, preds, color='red', label='Fitted')
plt.xlabel('Market Return')
plt.ylabel('Equity Return')
plt.title('CAPM Calibration')
plt.legend()
plt.show()

## Multi-Factor Model Calibration

In [ ]:
factors_list = ['market_return', 'smb', 'hml']  # Example: Fama-French 3 factor
X_multi = df[factors_list].values
y = df['equity_return'].values

model_multi = LinearRegression().fit(X_multi, y) # type: ignore
coeffs = dict(zip(factors_list, model_multi.coef_))

print("Factor loadings:")
for k,v in coeffs.items():
    print(f"  {k}: {v:.4f}")

## Save Calibrated Parameters

In [ ]:
params = {
    'alpha': float(alpha),
    'beta': float(beta),
    'multi_factor_loadings': coeffs,
}
pd.Series(params).to_json('./data/calibrated_params.json')
print("Saved calibrated parameters to ./data/calibrated_params.json")